In [19]:
import numpy as np
import pandas as pd
from collections import defaultdict
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
from gensim import 
import matplotlib.pyplot as plt
import operator

In [6]:
def TokenizedTexts(filename, min_freq):
    df = pd.read_csv(filename, index_col=0, header=None).rename({1 : 'text_stopped'}, axis=1).fillna('')
    texts_tokenized = list(df['text_stopped'].str.split())
    
    assert len(texts_tokenized) == len(df)
    
    frequency = defaultdict(int)

    for text in texts_tokenized:
        for token in text:
            frequency[token] += 1
            
    texts = [[token for token in text if frequency[token] > min_freq] for text in texts_tokenized]
    indices = list(df.index)
    
    assert len(texts) == len(df)
    
    return texts,indices

In [11]:
def Dictionary(tokenized_texts):
    return corpora.Dictionary(tokenized_texts)

In [12]:
def Corpus(dictionary, texts, tfidf=True):
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    if not tfidf:
        return corpus
    
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

In [120]:
tokenized_texts = TokenizedTexts('/home/vgautam/stopping/out/socc_articles.csv', 3)
dictionary = Dictionary(tokenized_texts)
corpus = Corpus(dictionary, tokenized_texts)

In [17]:
all_models = [models.ldamodel.LdaModel(corpus,
                                       id2word=dictionary,
                                       num_topics=15,
                                       chunksize=i,
                                       eval_every=1,
                                       iterations=200,
                                       passes=2)
             for i in range(1,200)]

In [50]:
lda_stats = []

for index in range(len(all_models)):
    lda = all_models[index]
    print('chunksize:', index+1)

    predictions = lda.get_document_topics(corpus)

    lda_topics = []

    for doc in predictions:
        lda_topics.append(max(doc, key=operator.itemgetter(1))[0])

    lda_df = pd.DataFrame(lda_topics, columns=['blah'])
    stddev = lda_df['blah'].value_counts().describe()['std']
    uniquevals = len(lda_df['blah'].unique())
    
    lda_stats.append([stddev, uniquevals])

chunksize: 1
chunksize: 2
chunksize: 3
chunksize: 4
chunksize: 5
chunksize: 6
chunksize: 7
chunksize: 8
chunksize: 9
chunksize: 10
chunksize: 11
chunksize: 12
chunksize: 13
chunksize: 14
chunksize: 15
chunksize: 16
chunksize: 17
chunksize: 18
chunksize: 19
chunksize: 20
chunksize: 21
chunksize: 22
chunksize: 23
chunksize: 24
chunksize: 25
chunksize: 26
chunksize: 27
chunksize: 28
chunksize: 29
chunksize: 30
chunksize: 31
chunksize: 32
chunksize: 33
chunksize: 34
chunksize: 35
chunksize: 36
chunksize: 37
chunksize: 38
chunksize: 39
chunksize: 40
chunksize: 41
chunksize: 42
chunksize: 43
chunksize: 44
chunksize: 45
chunksize: 46
chunksize: 47
chunksize: 48
chunksize: 49
chunksize: 50
chunksize: 51
chunksize: 52
chunksize: 53
chunksize: 54
chunksize: 55
chunksize: 56
chunksize: 57
chunksize: 58
chunksize: 59
chunksize: 60
chunksize: 61
chunksize: 62
chunksize: 63
chunksize: 64
chunksize: 65
chunksize: 66
chunksize: 67
chunksize: 68
chunksize: 69
chunksize: 70
chunksize: 71
chunksize: 72
c

In [53]:
lda_stats[14]

[1339.1948256143992, 15]

In [52]:
pd.DataFrame(lda_stats).sort_values(0, ascending=True)

,0,1
31,1216.536430,14
14,1339.194826,15
34,1393.357130,11
41,1406.926916,14
143,1463.216912,13
38,1482.930341,13
24,1490.608096,15
181,1508.767676,12
37,1529.435781,14
140,1531.136117,14


In [51]:
pd.DataFrame(lda_stats).sort_values(1, ascending=False)

,0,1
60,2194.278322,15
145,1605.530018,15
94,2102.466119,15
35,1700.909902,15
36,1740.900812,15
14,1339.194826,15
24,1490.608096,15
25,1861.120648,15
88,2021.667391,14
85,2113.198260,14


In [55]:
lda = all_models[14]

predictions = lda.get_document_topics(corpus)

lda_topics = []

for doc in predictions:
    lda_topics.append(max(doc, key=operator.itemgetter(1))[0])

lda_df = pd.DataFrame(lda_topics, columns=['blah'])
stddev = lda_df['blah'].value_counts().describe()['std']
uniquevals = len(lda_df['blah'].unique())

print(stddev, uniquevals)

1339.1948256143992 15


In [54]:
all_models[14].save('LDAfinal.model')

In [42]:
dictionary.save('socc_articles.dict')
corpora.MmCorpus.serialize('socc_articles.mm', corpus)

In [97]:
with open('socc_articles_topics.txt', 'w+') as fo:
    for i,doc in enumerate(predictions):
#         print(str(doc))
        fo.write(str(i) + ';' + str(doc))
        fo.write('\n')

In [100]:
with open('topics.txt', 'w+') as fo:
    for i in range(15):
        print(str(i) + ':', [k for k,v in lda.show_topic(i)])
        fo.write(str(i) + ': ' + str([k for k,v in lda.show_topic(i)]))

0: ['province', 'McGuinty', 'Drummond', 'gas', 'Duncan', 'point', 'budget', 'bill', 'game', 'premier']
1: ['Romney', 'school', 'local', 'life', 'personal', 'North', 'air', 'home', 'ministers', 'mother']
2: ['city', 'Ford', 'mayor', 'report', 'police', 'sector', 'work', 'Toronto', 'budget', 'costs']
3: ['NDP', 'Ontario', 'government', 'party', 'Harper', 'Mulcair', 'health', 'Liberals', 'spending', 'tax']
4: ['China', 'Santorum', 'Gingrich', 'leadership', 'Republican', 'trade', 'free', 'human', 'Republicans', 'House']
5: ['program', 'global', 'Charest', 'death', 'talk', 'Trade', 'legislation', 'wanted', 'apparently', 'Commons']
6: ['Redford', 'candidates', 'ago', 'problem', 'sex', 'matter', 'community', 'months', 'hold', 'kind']
7: ['aboriginal', 'natural', 'communities', 'protection', 'reserves', 'lead', 'reality', 'leading', 'Crown', 'suggest']
8: ['Obama', 'age', 'Internet', 'judge', 'Senate', 'Thomas', 'conservatives', 'raised', 'media', 'success']
9: ['public', 'people', 'system', '

In [125]:
tokenized_texts, indices = TokenizedTexts('/home/vgautam/stopping/out/socc_comments.csv', 1)
corpus = Corpus(dictionary, tokenized_texts)

predictions = lda[corpus]

assert len(predictions) == len(tokenized_texts)
assert len(tokenized_texts) == len(indices)

corpora.MmCorpus.serialize('socc_comments.mm', corpus)
# corpus = MmCorpus('topic_modelling/corpora/socc_comments.mm')

with open('socc_comments_topics.txt', 'w+') as fo:
    for i,doc in enumerate(predictions):
#         print(i,str(doc))
        fo.write(str(i) + ';' + str(doc))
        fo.write('\n')

In [126]:
tokenized_texts, indices = TokenizedTexts('/home/vgautam/stopping/out/tyee_comments.csv', 1)
corpus = Corpus(dictionary, tokenized_texts)

predictions = lda[corpus]

assert len(predictions) == len(tokenized_texts)
assert len(tokenized_texts) == len(indices)

corpora.MmCorpus.serialize('tyee_comments.mm', corpus)
# corpus = MmCorpus('topic_modelling/corpora/socc_comments.mm')

with open('tyee_comments_topics.txt', 'w+') as fo:
    for i,doc in enumerate(predictions):
#         print(i,str(doc))
        fo.write(str(i) + ';' + str(doc))
        fo.write('\n')

In [127]:
tokenized_texts, indices = TokenizedTexts('/home/vgautam/stopping/out/conv_comments.csv', 1)
corpus = Corpus(dictionary, tokenized_texts)

predictions = lda[corpus]

assert len(predictions) == len(tokenized_texts)
assert len(tokenized_texts) == len(indices)

corpora.MmCorpus.serialize('conv_comments.mm', corpus)
# corpus = MmCorpus('topic_modelling/corpora/socc_comments.mm')

with open('conv_comments_topics.txt', 'w+') as fo:
    for i,doc in enumerate(predictions):
#         print(i,str(doc))
        fo.write(str(i) + ';' + str(doc))
        fo.write('\n')